In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j_2.12/1.3.1/xgboost4j_2.12-1.3.1.jar
!wget https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark_2.12/1.3.1/xgboost4j-spark_2.12-1.3.1.jar


--2021-03-16 12:15:28--  https://repo1.maven.org/maven2/ml/dmlc/xgboost4j_2.12/1.3.1/xgboost4j_2.12-1.3.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3432273 (3.3M) [application/java-archive]
Saving to: ‘xgboost4j_2.12-1.3.1.jar.5’

xgboost4j_2.12-1.3. 100%[===================>]   3.27M  --.-KB/s    in 0.03s   

2021-03-16 12:15:28 (106 MB/s) - ‘xgboost4j_2.12-1.3.1.jar.5’ saved [3432273/3432273]

--2021-03-16 12:15:28--  https://repo1.maven.org/maven2/ml/dmlc/xgboost4j-spark_2.12/1.3.1/xgboost4j-spark_2.12-1.3.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276094 (270K) [application/java-archive]
Saving to: ‘xgboost4j-spark_2.12-1.3.1.j

In [ ]:
!ls
!pwd

diabetes.csv
drive
part-00000-61b3265b-867d-4c02-b03a-27a7c1b91e33-c000.snappy.parquet
sample_data
spark-2.4.7-bin-hadoop2.7
spark-2.4.7-bin-hadoop2.7.tgz
spark-2.4.7-bin-hadoop2.7.tgz.1
spark-2.4.7-bin-hadoop2.7.tgz.2
spark-2.4.7-bin-hadoop2.7.tgz.3
spark-2.4.7-bin-hadoop2.7.tgz.4
sparkxgb.zip
xgboost4j_2.12-1.3.1.jar
xgboost4j_2.12-1.3.1.jar.1
xgboost4j_2.12-1.3.1.jar.2
xgboost4j_2.12-1.3.1.jar.3
xgboost4j_2.12-1.3.1.jar.4
xgboost4j_2.12-1.3.1.jar.5
xgboost4j-spark_2.12-1.3.1.jar
xgboost4j-spark_2.12-1.3.1.jar.1
xgboost4j-spark_2.12-1.3.1.jar.2
xgboost4j-spark_2.12-1.3.1.jar.3
xgboost4j-spark_2.12-1.3.1.jar.4
xgboost4j-spark_2.12-1.3.1.jar.5
/content


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/xgboost4j_2.12-1.3.1.jar,/content/xgboost4j-spark_2.12-1.3.1.jar,/content/sparkxgb.zip pyspark-shell'

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.addPyFile("/content/sparkxgb.zip")

In [ ]:
reader = spark.read

In [ ]:
from sparkxgb import XGBoostEstimator

import numpy as np
import pandas as pd

In [ ]:
train_data = spark.read.csv("/content/diabetes.csv",header=True)

In [ ]:
train_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [ ]:
from pyspark.sql.functions import col
new_data = train_data.select(*(col(c).cast("float").alias(c) for c in train_data.columns))

In [ ]:
from pyspark.sql.functions import count, isnan, when
#checking for null ir nan type values in our columns
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
cols=new_data.columns
cols.remove("Outcome")
assembler = VectorAssembler(inputCols=cols,outputCol="features")

# Now let us use the transform method to transform our dataset
data=assembler.transform(new_data)

data.select("features",'Outcome').show(truncate=False)

+-----------------------------------------------------------------------+-------+
|features                                                               |Outcome|
+-----------------------------------------------------------------------+-------+
|[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |1.0    |
|[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0]   |0.0    |
|[8.0,183.0,64.0,0.0,0.0,23.299999237060547,0.671999990940094,32.0]     |1.0    |
|[1.0,89.0,66.0,23.0,94.0,28.100000381469727,0.16699999570846558,21.0]  |0.0    |
|[0.0,137.0,40.0,35.0,168.0,43.099998474121094,2.2880001068115234,33.0] |1.0    |
|[5.0,116.0,74.0,0.0,0.0,25.600000381469727,0.20100000500679016,30.0]   |0.0    |
|[3.0,78.0,50.0,32.0,88.0,31.0,0.24799999594688416,26.0]                |1.0    |
|[10.0,115.0,0.0,0.0,0.0,35.29999923706055,0.1340000033378601,29.0]     |0.0    |
|[2.0,197.0,70.0,45.0,543.0,30.5,0.15800000727176666,53.0]              |1.0    |
|[8.0,125.0,96.0

In [ ]:
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
data=standardscaler.fit(data).transform(data)

In [ ]:
data.select("features",'Outcome','Scaled_features').show(truncate=False)

+-----------------------------------------------------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                               |Outcome|Scaled_features                                                                                                                                          |
+-----------------------------------------------------------------------+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |1.0    |[1.7806383732194306,4.628960915766174,3.7198138711154307,2.1940523222807116,0.0,4.261709202425419,1.8923810993699686,4.251616970894646]                  |
|[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.3510000109

In [ ]:
assembled_data = data.select("Scaled_features","Outcome")
assembled_data.show()

+--------------------+-------+
|     Scaled_features|Outcome|
+--------------------+-------+
|[1.78063837321943...|    1.0|
|[0.29677306220323...|    0.0|
|[2.37418449762590...|    1.0|
|[0.29677306220323...|    0.0|
|[0.0,4.2849165233...|    1.0|
|[1.48386531101619...|    0.0|
|[0.89031918660971...|    1.0|
|[2.96773062203238...|    0.0|
|[0.59354612440647...|    1.0|
|[2.37418449762590...|    1.0|
|[1.18709224881295...|    0.0|
|[2.96773062203238...|    1.0|
|[2.96773062203238...|    0.0|
|[0.29677306220323...|    1.0|
|[1.48386531101619...|    1.0|
|[2.07741143542266...|    1.0|
|[0.0,3.6906580274...|    1.0|
|[2.07741143542266...|    1.0|
|[0.29677306220323...|    0.0|
|[0.29677306220323...|    1.0|
+--------------------+-------+
only showing top 20 rows



In [ ]:
from sparkxgb import XGBoostEstimator

xgboost = XGBoostEstimator(
    featuresCol="features", 
    labelCol="Survival", 
    predictionCol="prediction"
)

TypeError: ignored

In [ ]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 2
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

In [ ]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

In [ ]:
!nvidia-smi

In [ ]:
model.write().overwrite().save('/content/model/')

In [ ]:
!ls

In [ ]:
loaded_model = XGBoostClassificationModel().load('/content/model/')

In [ ]:
result=loaded_model.transform(test_data)

In [ ]:
result.select(label, 'rawPrediction', 'probability', 'prediction').show(5)

In [ ]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)